# Retrieval Augmented Generative Question Answer Using OCI OpenSearch as Retriever

In this tutorial, we will walk through the steps to set up a retrieval-augmented generative QA using OCI OpenSearch as retriever.

### Prerequesites
- You have a Running Instance of OCI Search.
- OpenSearch version has to be at least 2.8.0.
- You need to install langchain, opensearch-py, and oracle-ads.

To check how to spin up an instance of OCI search, see [Search and visualize data using OCI Search Service with OpenSearch](https://docs.oracle.com/en/learn/oci-opensearch/index.html#introduction)

In [ ]:
!pip install --upgrade oracle-ads langchain opensearch-py

### Step 1: Load and Split your Documents Into Chunks
Let's say you're looking to create a search engine that enables users to search through documentation stored as markdown files. Actually, it does not matter what file format your documentation are in as Langchain offers support for various types of document loaders. In this tutorial, we will just use markdown file as an example.

In [1]:
import fsspec
from langchain.text_splitter import MarkdownHeaderTextSplitter

with fsspec.open(
    "https://raw.githubusercontent.com/oracle-samples/oci-data-science-ai-samples/main/distributed_training/Tensorboard.md",
    "r"
) as f:
    report = f.read()
    
    
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
md_header_splits = markdown_splitter.split_text(report)
texts = [text.page_content for text in md_header_splits]

In [2]:
print(f"Number of documents: {len(texts)}")
print(f"First document:\n{texts[0]}")

Number of documents: 4
First document:
TensorBoard helps visualizing your experiments. You bring up a ``TensorBoard`` session on your workstation and point to
the directory that contains the TensorBoard logs.  
`OCI` = Oracle Cloud Infrastructure
`DT` = Distributed Training
`ADS` = Oracle Accelerated Data Science Library
`OCIR` = Oracle Cloud Infrastructure Registry


### Step 2: Embed your Documents

You can use oracle-ads to access the GenerativeAI embedding models. The embedding models returns embedding vectors of length 1024. oracle-ads is an open source library. It speeds up common data science activities by providing tools that automate and simplify common data science tasks. Additionally, provides data scientists a friendly pythonic interface to OCI services. Check [oracle-ads github](https://github.com/oracle/accelerated-data-science) for more information.

In [ ]:
from ads.llm import GenerativeAIEmbeddings
 
oci_embedings = GenerativeAIEmbeddings(
    compartment_id="ocid1.compartment.oc1.######",
    client_kwargs=dict(service_endpoint="https://generativeai.aiservice.us-chicago-1.oci.oraclecloud.com") # this can be omitted after Generative AI service is GA.
)
embeddings = oci_embedings.embed_documents(texts=texts)

In [4]:
print(f"Number of embeddings: {len(embeddings)}")
print(f"Embedding dimensions: {len(embeddings[0])}")

Number of embeddings: 4
Embedding dimensions: 1024


### Step 3: Create an Index for your Documents

First connect to your OCI search cluster. We can use the opensearchpy library to connect to the OpenSearch cluster.

In [6]:
# Connect to the opensearch cluster.
from opensearchpy import OpenSearch
 
# Create a connection to your OpenSearch cluster
es = OpenSearch(
    ['https://####'],  # Replace with your OpenSearch endpoint URL
    http_auth=('username', 'password'),  # Replace with your credentials
    verify_certs=False,  # Set to True if you want to verify SSL certificates
    timeout=30
)

First, you must create a k-NN index and set the ``index.knn`` parameter to true. This settings tells the plugin to generate native library indexes specifically tailored for k-NN searches. 

Next, you must add one or more fields of the knn_vector data type. This example creates an index with one ``knn_vector``: ``embedding_vector`` and one ``text``: ``text``. 

The knn_vector uses Lucene fields that specify the configuration of the k-NN search algorithms. It employs the Hierarchical Navigable Small Worlds [HNSW](https://www.pinecone.io/learn/series/faiss/hnsw/) algorithm  for super fast search and fantastic recall and consine similarity to measure distance. 

- ``efSearch`` controls how many entry points will be explored between layers during the search. A higher value of ef_search typically results in a more thorough and potentially higher-quality search, but increased computational cost. 

- ``efConstruction`` controls how many entry points will be explored when building the index. A higher value of "ef_constructions" typically results in a higher-quality graph structure but may also increase the computational cost of building the index.

The ``dimension`` field defines the size of the embedding vector. In our case, we are using embedding vectors returned from the genAI embedding model, which is of length 1024. 

See [documentation](https://opensearch.org/docs/2.8/search-plugins/knn/knn-index#method-definitions) for more details on parameters' definitions. You

**Note**: The Lucene engine can support dimension up to 1,024.

In [7]:
INDEX_NAME = "tensorboard"
VECTOR_1_NAME = "embedding_vector"
VECTOR_2_NAME = "text"
 
body = {
    # Index setting: https://opensearch.org/docs/2.11/search-plugins/knn/knn-index
    "settings": {"index": {"knn": "true", "knn.algo_param.ef_search": 100}},
    # Explicit mapping: https://opensearch.org/docs/2.11/field-types/index/#explicit-mapping
    "mappings": { 
        "properties": {
            VECTOR_1_NAME: {
                # Supported field types: https://opensearch.org/docs/2.11/field-types/supported-field-types/index/
                "type": "knn_vector", 
                "dimension": 1024,
                # Method definition: https://opensearch.org/docs/2.11/search-plugins/knn/knn-index#method-definitions
                "method": { 
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene",
                    "parameters": {"ef_construction": 128, "m": 24},
                },
            },
            VECTOR_2_NAME: {
                 "type": "text"
               },
        }
    },
}
response = es.indices.create(INDEX_NAME, body=body)
response

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tensorboard'}

### Step 4: Insert the Embedding Vectors for your Documents
Now let's populate the index using the embedding vectors calculated from your documents using Cohere Embedding Models. 

In [8]:
i = 0
# insert each row one-at-a-time to the document index
for text, embed in zip(texts, embeddings):
 
    try:
         
        body = {
            VECTOR_1_NAME: embed,
            VECTOR_2_NAME: text,
        }
        response = es.index(index=INDEX_NAME, body=body)
    except Exception as e:
        print(f"[ERROR]: {e}")
        continue
    i += 1


A new query coming in, first calcualte the embedding vector and then conduct a semantic search.

- `k`: the number of neighbors the search will return
- `size`: (required) how many results the query actually returns. The plugin returns k amount of results for each shard (and each segment) and size amount of results for the entire query. The plugin supports a maximum k value of 10,000.

In [15]:
query_vector = oci_embedings.embed_query(text="how to set up tensorboard in oci?")
query = {
    "size": 2,
    "query": {"knn": {VECTOR_1_NAME: {"vector": query_vector, "k": 2}}},
}
 
response = es.search(body=query, index=INDEX_NAME)  # the same as before
print(response["hits"]["hits"][0]['_source']['text'])

It is required that ``tensorboard`` is installed in a dedicated conda environment or virtual environment. Prepare an
environment yaml file for creating conda environment with following command -  
**tensorboard-dep.yaml**:  
```yaml
dependencies:
- python=3.8
- pip
- pip:
- ocifs
- tensorboard
name: tensorboard
```  
Create the conda environment from the yaml file generated in the preceeding step  
```bash
conda env create -f tensorboard-dep.yaml
```  
This will create a conda environment called tensorboard. Activate the conda environment by running -  
```bash
conda activate tensorboard
```  
**Using TensorBoard Logs:**  
To launch a TensorBoard session on your local workstation, run -  
```bash
export OCIFS_IAM_KEY=api_key
tensorboard --logdir oci://my-bucket@my-namespace/path/to/logs
```  
`OCIFS_IAM_KEY=api_key` - If you are using resource principal, set `resource_principal`  
This will bring up TensorBoard app on your workstation. Access TensorBoard at ``http://localhost:6006/``  